In [4]:
import networkx as nx
import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf
from itertools import product
from sklearn.model_selection import train_test_split
from copy import deepcopy

In [ ]:
y_any = y_pred.apply(np.mean, axis = 1).apply(int)
evaluation(y_test, y_any)

In [ ]:
y_prob_train = pd.DataFrame(
    {'lr': [x[1] for x in clf_lr.predict_proba(X_train)],
     'rf': [x[1] for x in clf_rf.predict_proba(X_train)],
     'svm': [x[1] for x in clf_svm.predict_proba(X_train)],
     'ada': [x[1] for x in clf_ada.predict_proba(X_train)],
     'nn': [x[0] for x in clf_nn.predict(X_train)]
})
y_prob_train.head(n = 10)

In [ ]:
y_mean = y_prob_test.apply(np.mean, axis = 1); y_mean.head(n = 10)

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt 
fpr, tpr, thresholds = roc_curve(y_test, y_mean)
youden = tpr - fpr
youden_fpr = fpr[np.argmax(youden)]
youden_tpr = tpr[np.argmax(youden)]
cutoff = thresholds[np.argmax(youden)]  
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve')
plt.plot([youden_fpr, youden_fpr], [youden_fpr, youden_tpr],  color='black', lw=lw / 2, linestyle='--',
         label='Max Youden index (cutoff = {:.2f})'.format(cutoff))
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
y_final =  1 * (y_mean > cutoff)
evaluation(y_test, y_final)

In [ ]:
meta_train = y_prob_train.values; len(meta_train)

In [ ]:
# Neural network
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD

N_features = meta_train.shape[1]
BATCH_SIZE = 128

meta = Sequential()
meta.add(Dense(N_features, activation='relu', use_bias = True))
model.add(Dropout(0.5))
meta.add(Dense(1, activation='sigmoid', use_bias=True))
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
meta.compile(optimizer=sgd,loss='binary_crossentropy',metrics=['accuracy'])

# Fit the meta
meta.fit(meta_train, y_train, epochs=20, batch_size=BATCH_SIZE)

In [ ]:
meta_test = y_prob_test.values
meta_pred = meta.predict_classes(meta_test)
evaluation(y_test, meta_pred)

In [ ]:
non_stand_sim_test = similarity(G_master, test_pairs); non_stand_sim_test.head()

In [ ]:
sim_test = scaler.transform(non_stand_sim_test); sim_test

In [ ]:
final_test = pd.DataFrame(
    {'lr': [x[1] for x in clf_lr.predict_proba(sim_test)],
     'rf': [x[1] for x in clf_rf.predict_proba(sim_test)],
     'svm': [x[1] for x in clf_svm.predict_proba(sim_test)],
     'ada': [x[1] for x in clf_ada.predict_proba(sim_test)],
     'nn': [x[0] for x in clf_nn.predict(sim_test)]
})
final_test.head(n = 10)

In [ ]:
final_mean = final_test.apply(np.mean, axis = 1)
final_pred = 1 * (final_mean > cutoff); final_pred

In [ ]:
row_list = []
for i in range(2000):
    pair = test_pairs[i]
    row = {}
    row['Id'] = i + 1
    row['Predictions'] = final_pred[i]
    row_list.append(row)

predictions = pd.DataFrame(row_list)
predictions.to_csv('predictions.csv', index=False)